<a href="https://colab.research.google.com/github/vamshitn/Samsung-innovation-campus/blob/main/intro_to_unsupervised_learning_with_kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a name='0'></a>
# Intro to Unsupervised Learning - K-Means Clustering

K-Means clustering is a type of unsupervised learning algorithms. In unsupervised learning, the machine learning model do not get the labels during training. It instead has to figure out the labels itself. It's like learning without instructions. It's like a teacher telling you, "hey, here are 1000 exercises to use while preparing for a test, the test will be only 5 questions from all of those exercises." That can feel like a struggle, you will do all you can to narrow down these 100 exercises to 5. Some questions may be similar, or may be solved by one method, etc..the goal will merely be to narrow down the exercises, while maximizing the chance of passing the test.

That type of example can be compared to clustering. The model is given bunch of data (lacking labels) and the job of the model is to find the labels that can be present according to the supplied data.


K-Means Clustering require the number of clusters to be specified before training. The way this type of algorithm works is beyond the scope of this notebook but here are 3 main steps of how such algorithm work:

* K-Means will randomly assigns samples of data to initial centroids of all clusters. This step is called initialization. A centroid is also referred to as a cluster center and it is the mean of all the sample of data in a cluster.

* It then reassigns the samples to the nearest centroids.
* It also find the new centroids of all clusters by taking the mean value of all of the samples assigned to each previous centroids.

The last two steps are repeated until the stopping criterion is fulfilled or when difference between the old and new centroids is constant.

Unspervised learning has got its applications in areas such as grouping web search results, customer segmentation, news aggregation and more.

## KMeans Clustering

### Contents

* [1 - Imports](#1)
* [2 - Loading the data](#2)
* [3 - Exploratory Analysis](#3)
* [4 - Preprocessing the data](#4)
* [5 - Trainin K-Means Clustering to Find Clusters](#5)
* [6 - Evaluating K-Means Clustering](#6)
* [7 - Final Notes](#7)

<a name='1'></a>
## 1 - Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

<a name='2'></a>

## 2 - Loading the data

In this notebook, we will use a different dataset. Up to this point creating these notebooks, my goal has been to look on the other side, to try something new, to try new a dataset. If you have went through some notebooks about other algorithms, no doubt that you have learned something new or perhaps nothing new but you experienced a new dataset.

In this notebook, we will use a mushroom dataset. The dataset describes mushrooms in terms of their physical characteristics and they are classified into: poisonous or edible.

The dataset also includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family. Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like `leaflets three, let it be for Poisonous Oak and Ivy.`

The dataset contains the labels (edibility) but for the purpose of doing clustering, we will remove the labels.

In [ ]:
# Let's firs hide warnings just in case

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# from sklearn.datasets import fetch_openml

# mushroom_data = fetch_openml(name='mushroom', version=1)

In [ ]:
# Fallback to downloading the dataset from a URL if fetch_openml fails
import requests
import io
import os
import pandas as pd
url = "https://raw.githubusercontent.com/ustunb/risk-slim/refs/heads/master/examples/data/mushroom_data.csv"
# Read the content directly as a CSV file
mushroom_data = pd.read_csv(url)

In [ ]:
mushroom_data.shape

As you can see above, there are 8124 examples and 22 features.

In [ ]:
# Displaying feature names

mushroom_data.info()

In [ ]:
# Displaying target name

mushroom_data.columns

<a name='3'></a>
## 3 - Exploratory Data Analysis


### Taking a quick look into the dataset

In [ ]:
mushroom_data.head()

In [ ]:
# Displaying the last rows

mushroom_data.tail()

All features are categorical. So we will make sure to handle them.

### Checking Summary Statistics

In [ ]:
# Summary stats

mushroom_data.describe()

### Checking Missing Values

In [ ]:
# Checking missing values

mushroom_data.isnull().sum()

It seems that we have missing values in the feature `stalk-root`.

Usually there are three things to do with if them if they are present:
* We can remove all missing values completely
* We can leave them as they are or
* We can fill them with a given strategy such as mean, media or most frequent value. Either `Sklearn` or Pandas provides a quick ways to fill these kind of values.

We will handle that during the data preprocessing.

### More Data Exploration

Before preprocessing the data, let's take a look into specific features.

I want to also make note that I do not know alot about mushrooms. I thought that it would be interesting to use this real world datasets, and perhaps some people who will come across this may some of mushroom samples and their characteristics.

In [ ]:
# Identify columns related to cap shape
cap_shape_cols = [col for col in mushroom_data.columns if 'cap_shape_eq_' in col]

# Sum the one-hot encoded columns to get counts for each cap shape
cap_shape_counts = mushroom_data[cap_shape_cols].sum().reset_index()
cap_shape_counts.columns = ['cap_shape', 'count']

# Extract the actual cap shape name from the column name
cap_shape_counts['cap_shape'] = cap_shape_counts['cap_shape'].str.replace('cap_shape_eq_', '')

plt.figure(figsize=(12,7))
sns.barplot(data=cap_shape_counts, x='cap_shape', y='count')
plt.title('Distribution of Mushroom Cap Shapes')
plt.xlabel('Cap Shape')
plt.ylabel('Count')
plt.show()

In cap_shape, the letters stands for: `ell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s`. It seems that the convex type is dominant and most of it are edible.

In [ ]:
# Identify columns related to cap color and the poisonous column
cap_color_cols = [col for col in mushroom_data.columns if 'cap_color_eq_' in col]
poisonous_col = 'poisonous'

# Combine the cap color columns and the poisonous column
cap_color_data = mushroom_data[cap_color_cols + [poisonous_col]]

# Melt the DataFrame to long format to easily plot the distribution of cap colors by poisonous status
cap_color_melted = cap_color_data.melt(id_vars=[poisonous_col], var_name='cap_color', value_name='is_present')

# Filter out rows where the cap color is not present for a given mushroom
cap_color_melted = cap_color_melted[cap_color_melted['is_present'] == 1]

# Extract the actual cap color name from the column name
cap_color_melted['cap_color'] = cap_color_melted['cap_color'].str.replace('cap_color_eq_', '')

plt.figure(figsize=(15, 8))
sns.countplot(data=cap_color_melted, x='cap_color', hue=poisonous_col)
plt.title('Distribution of Mushroom Cap Colors by Poisonous Status')
plt.xlabel('Cap Color')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

The above is the cap color. The alphabets stands for `brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y `.

Also it seems that most caps are brown(n), either edible or brown.

In [ ]:
# Identify columns related to population
population_cols = [col for col in mushroom_data.columns if 'population_eq_' in col]

# Sum the one-hot encoded columns to get counts for each population type
population_counts = mushroom_data[population_cols].sum().reset_index()
population_counts.columns = ['population', 'count']

# Extract the actual population type name from the column name
population_counts['population'] = population_counts['population'].str.replace('population_eq_', '')

plt.figure(figsize=(12,7))
sns.barplot(data=population_counts, x='population', y='count')
plt.title('Distribution of Mushroom Population Types')
plt.xlabel('Population Type')
plt.ylabel('Count')
plt.show()

The most populations are most several. Here are what the letters stand for: `abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y`.  

In [ ]:
# Identify columns related to habitat
habitat_cols = [col for col in mushroom_data.columns if 'habitat_eq_' in col]

# Sum the one-hot encoded columns to get counts for each habitat type
habitat_counts = mushroom_data[habitat_cols].sum().reset_index()
habitat_counts.columns = ['habitat', 'count']

# Extract the actual habitat type name from the column name
habitat_counts['habitat'] = habitat_counts['habitat'].str.replace('habitat_eq_', '')

plt.figure(figsize=(12,7))
sns.barplot(data=habitat_counts, x='habitat', y='count')
plt.title('Distribution of Mushroom Habitat Types')
plt.xlabel('Habitat Type')
plt.ylabel('Count')
plt.show()

In [ ]:
# Identify columns related to stalk root
stalk_root_cols = [col for col in mushroom_data.columns if 'stalk_root_eq_' in col]

# Sum the one-hot encoded columns to get counts for each stalk root type
stalk_root_counts = mushroom_data[stalk_root_cols].sum().reset_index()
stalk_root_counts.columns = ['stalk_root', 'count']

# Extract the actual stalk root type name from the column name
stalk_root_counts['stalk_root'] = stalk_root_counts['stalk_root'].str.replace('stalk_root_eq_', '')

plt.figure(figsize=(12,7))
sns.barplot(data=stalk_root_counts, x='stalk_root', y='count')
plt.title('Distribution of Mushroom Stalk Root Types')
plt.xlabel('Stalk Root Type')
plt.ylabel('Count')
plt.show()

Above is the feature that is missing values. We will remove all missing values. Since the missing values are of one category, we will drop it to avoid adding noise in the dataset.

And finally, we can look in the class feature. There are two categories, `e(edible)` and `p(poisonous)`.

In [ ]:
plt.figure(figsize=(12,7))

sns.countplot(data=mushroom_data, x='poisonous')

<a name='4'></a>

## 4 - Data Preprocessing


Let's remove the missing values first.

In [ ]:
mushroom_df = mushroom_data.dropna()

For the purpose of performing clustering, we will remove the labels.

In [ ]:
mushroom_df.columns

In [ ]:
mushroom_data['poisonous'].value_counts()

In [ ]:
mushroom = mushroom_df.drop('poisonous', axis=1)
mushroom_labels = mushroom_df['poisonous']

Let's now convert all categorical features into the numerics.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

mushroom_prepared = encoder.fit_transform(mushroom)

In [ ]:
mushroom_prepared

As you can see above, `mushroom_prepared` is a NumPy array. We can convert it back to the Pandas Dataframe although KMeans algorithm can accept both as input.

In [ ]:
mushroom_prep_df = pd.DataFrame(mushroom_prepared, columns=mushroom.columns)
mushroom_prep_df.head()

No alphabets anymore. They were perfectly encoded or converted to numerics representation.

We are now ready to find the labels with KMeans Clustering. Again, this is for the assumption that we do not have labels, or to make it simple, we have a data about the characteristics of different plants, but we do not know if they are edible or not. We want to use unsupervised learning to figure that out.

<a name='5'></a>

## 5 - Training K-Means Clustering to Find Clusters

We are going to create a KMeans model from `sklearn.cluster`. We will remember to provide the number of the clusters, which is 2 in our case.

In [ ]:
from sklearn.cluster import KMeans

k_clust = KMeans(n_clusters=2, random_state=42)

k_clust.fit(mushroom_prep_df)

We can access the cluster centers by `model.cluster_centers_`.

In [ ]:
k_clust.cluster_centers_

Also, we can get the labels that the KMeans provided for each data point.

In [ ]:
k_labels = k_clust.labels_
k_labels

<a name='6'></a>

### 6 -Evaluating K-Means Clustering

In real world, evaluating the performance of KMeans is not an easy thing, because there are not true labels to compare with the clustered labels. In our case since we have them, we can find things like accuracy score, or even find the confusion matrix to display the actual and predicted classes. Not to mention classification report to find things like Recall, Precision, or F1 Score.

But again since we are merely comparing the labels(true and clustered), we do not need that extra metrics.

Before finding the accuracy score, I will first convert the true labels into the numbers or encode them. For simplicity, I will use a map function.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(mushroom_labels, k_labels)

This is not excellent, but it's so impressive. Why? Well, KMeans never saw the labels, it was only feed the data of different characteristics of poisonous and edible mushrooms and its job was to try to find patterns in the data so as to learn if a given mushroom specy is a poisonous or edible.


KMeans algorithm is very useful in areas where you have a bunch of unlabeled data. Take an example in customer segmentation. You may want to provide different promotions to some groups of your customers but you have no clue of who would benefit from that particular promotion. So, you can try to find the group of customers using this algorithm. It will try to group similar customers according to their interests, and will likely appreciate the promotion.

The same concept can be applied to grouping the equipments that has similar defects in an industry. That was just mentioning few, there are more applications of KMeans clustering.

<a name='7'></a>

### 7 - Final Notes

In this notebook, we learned the idea behind unsupervised learning and KMeans clustering. We also practiced that on mushroom dataset where we were interested in grouping the species that can be poisonous or edible.

If you like mushrooms and you know some of their characteristics, no doubt that you enjoyed this notebook. Maybe pick one edible sample and make it your next meal :)